# Resynchronizing Jittery AES Power Traces


What happens if things aren't as clean as we made them out to be? While we can use preprocessing modules!

## Capturing Jittery Traces

### Rebuilding New Firmware

In file `chipwhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes.c` find this:

```c
uint8_t get_pt(uint8_t* pt)
{
	trigger_high();
	aes_indep_enc(pt); /* encrypting the data block */
	trigger_low();
	simpleserial_put('r', 16, pt);
	return 0x00;
}
```

and add some random delay:

```c
uint8_t get_pt(uint8_t* pt)
{
	trigger_high();
       for(volatile uint8_t k = 0; k < (*pt & 0x0F); k++);
	aes_indep_enc(pt); /* encrypting the data block */
	trigger_low();
	simpleserial_put('r', 16, pt);
	return 0x00;
}
```

This deterministic delay is NOT a good countermeasure, but is much easier to write in a single line since we don’t have a CSPRNG linked in. We’ll break the jitter without relying on the deterministic aspect though, so our attack would work against a better jitter source.

**Be sure to remove this function afterwards so you don't break your code!**

We can build the code (change the platform as needed), and confirm the output of the following works as you expect:

In [ ]:
%%bash
cd ../../hardware/victims/firmware/
mkdir -p simpleserial-aes-lab1 && cp -r simpleserial-aes/* $_

In [ ]:
%%bash
cd ../../hardware/victims/firmware/simpleserial-aes-lab1
awk -i inplace '{gsub(/trigger_high\(\);/, "trigger_high\(\);\n\tfor\(volatile uint8_t k = 0; k < \(*pt \\& 0x0F\); k++\);\n")}1' simpleserial-aes.c 2>/dev/null

In [ ]:
%%bash
cd ../../hardware/victims/firmware/simpleserial-aes-lab1
make PLATFORM=CWLITEARM CRYPTO_TARGET=TINYAES128C

### Setup

Now let's go ahead. We'll have to program the file we built, so be sure to confirm we are using the right file!

In [ ]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [ ]:
import os, time

# uncomment based on your target
#%run "Helper_Scripts/Program_XMEGA.ipynb"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes-lab1/simpleserial-aes-CWLITEARM.hex"

modtime = os.path.getmtime(fw_path)
print("File build time: {:s} (built {:.2f} mins ago)".format(str(time.ctime(modtime)), (time.time() - modtime)/60.0))

In [ ]:
# program the target
program_target(scope, fw_path)

In addition, before we capture our traces, we'll need to create a ChipWhipserer project, since that's what Analyzer expects for an input:

In [ ]:
project = cw.createProject("projects/jupyter_test_jittertime", overwrite = True)

And we can get the class used to hold our traces by:

In [ ]:
from datetime import datetime
import copy

tc = copy.copy(project.getTraceFormat())
starttime = datetime.now()
prefix = starttime.strftime('%Y.%m.%d-%H.%M.%S') + "_"
tc.config.setConfigFilename(project.datadirectory + "traces/config_" + prefix + ".cfg")
tc.config.setAttr("prefix", prefix)
tc.config.setAttr("date", starttime.strftime('%Y-%m-%d %H:%M:%S'))

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and our new trace into our trace class. We'll also keep track of our keys manually for checking our answer later.

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

keys = []
N = 50  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    #traces.append(scope.getLastTrace())
    tc.addTrace(scope.getLastTrace(), text, "", key)

Now that we have our traces, we need to tell the project that the traces are loaded and add them to the project's trace manager.

In [ ]:
#Calling closeAll() to ensure flush to disk happens
tc.closeAll()

project.traceManager().appendSegment(tc)

#Save project file
project.save()

We're now done with the ChipWhisperer hardware, so we should disconnect from the scope and target:

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

Now that we have our traces, we can begin our attack! We'll start off by importing everything we need for the attack:

In [ ]:
from chipwhisperer.analyzer.attacks.cpa import CPA
from chipwhisperer.analyzer.attacks.cpa_algorithms.progressive import CPAProgressive
from chipwhisperer.analyzer.attacks.models.AES128_8bit import AES128_8bit, SBox_output
from chipwhisperer.analyzer.preprocessing.add_noise_random import AddNoiseRandom

Next, we'll add our traces to a preprocessing module. We can feed `project.traceManager()` right into `attack.setTraceSource()`, but we could also add pre-processing inbetween (more about this later). We'll also re-open the traces, in this case it is required since the call to `closeAll()` would have flushed the buffers.

In [ ]:
#Force reload of project data (if you comment out 'closeAll()' this isn't needed)

#We also rebuild the project object in case you only want to run this half
import chipwhisperer as cw
project = cw.openProject("projects/jupyter_test_jittertime")

This time we're going to do a few things. First we will get the traces, and plot a few of them as-is. You can adjust the traces plotted by adjusting the `range(10)`. For example `range(1)` plots the first trace.

In [ ]:
tm = project.traceManager()

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_5 as palette
import itertools  

output_notebook()
p = figure(sizing_mode='scale_width', plot_height=300)

# create a color iterator
colors = itertools.cycle(palette)  

x_range = range(0, tm.numPoints())
for i, color in zip(range(10), colors): #Adjust range(n) to plot certain traces
    p.line(x_range, tm.getTrace(i), color=color)
show(p)

So how do we fix that? To begin with, you should plot only a single trace to make your life more clear. You'll need to figure out a very unique area. For example see the following figure showing a single plot. In this example the location of **A** is unique, but **B** would have many matches within that same trace, even nearby:
![Resync example trace](img/resync_goodbad.png)

We will specify two items:
- A window with the "unique" area defined.
- How far we will shift the window (+/- points) to search for the best match.

You can use the following code to define the `target_window` and `max_shift`. Try a few values until you find something that works. Also try some poor example, and also try plotting more traces to confirm your match is working in real life.

In [ ]:
from chipwhisperer.analyzer.preprocessing.resync_sad import ResyncSAD

resync_traces = ResyncSAD(tm, connectTracePlot=False)
resync_traces.enabled = True
resync_traces.ref_trace = 0

#Define a target window here. 500,900 for example is good based on above. But try some different values.
resync_traces.target_window = (700, 1500)

# Define max_shift. Must not cause target_window to go outside of valid data. Try 16-600 range. Ideal value varies with how
# much jitter is in original data. 
resync_traces.max_shift = 700

#Uses objects from previous cells (plotting etc), so 
output_notebook()
p = figure()

for i, color in zip(range(10), colors):
    p.line(x_range, resync_traces.getTrace(i), color=color)
show(p)

preprocessed_traces = resync_traces

If this all works - let's just continue the attack! Do so as below:

In [ ]:
attack = CPA()
N = 50 #number of traces

leak_model = AES128_8bit(SBox_output)
attack.setAnalysisAlgorithm(CPAProgressive, leak_model)
attack.setTraceSource(preprocessed_traces, addToList=True) ##THIS LINE CHANGED TO INPUT TRACES
attack.setTraceStart(0)
attack.setTracesPerAttack(project.traceManager().numTraces())
attack.setIterations(1)
attack.setReportingInterval(10)
attack.setTargetSubkeys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
attack.setPointRange((0, -1))

And then actually run it:

In [ ]:
import pandas as pd
key = project.traceManager().getKnownKey(0)
def format_stat(stat):
    return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

def color_corr_key(row):
    global key
    ret = [""] * 16
    for i,bnum in enumerate(row):
        if bnum[0] == key[i]:
            ret[i] = "color: red"
        else:
            ret[i] = ""
    return ret

from IPython.display import clear_output
import numpy as np
        
def stats_callback():
    attack_results = attack.getStatistics()
    attack_results.setKnownkey(key)
    stat_data = attack_results.findMaximums()
    df = pd.DataFrame(stat_data).transpose()
    clear_output(wait=True)
    display(df.head().style.format(format_stat).apply(color_corr_key,axis=1))
    
attack_results = attack.processTracesNoGUI(stats_callback)

You *should* see the PGE reach 0 for each byte. If not, you might need to adjust the SAD resync. You could also need to increase the length of the sample capture for example. You may notice that it starts working OK and then fails, due to later traces become unsychronized.

### Plotting Correlation Output

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

attack_results = attack.getStatistics()
plot_data = cw.analyzerPlots(attack_results)
bnum = 0

ret = plot_data.outputVsTime(bnum)

output_notebook()
p = figure()
p.line(ret[0], ret[2], line_color='green')
p.line(ret[0], ret[3], line_color='green')

p.line(ret[0], ret[1], line_color='red')
show(p)

You should see a graph of red and green in time (samples). In red is the correlation of the correct subkey for the first byte, while the rest are in green.

You should see two or three distinctive red spikes. The first is the spot where the sbox lookup for the subkey we guessed actually happens (the later ones are from later steps in the AES operation).

What about the rest of the bytes in the key? We can get and plot that easily as well:

In [ ]:
rets = []
for i in range(0, 16):
    rets.append(plot_data.outputVsTime(i))

p = figure()
for ret in rets:
    p.line(ret[0], ret[2], line_color='green')
    p.line(ret[0], ret[3], line_color='green')
    
for ret in rets:
    p.line(ret[0], ret[1], line_color='red')

show(p)

## Conclusion

Awesome! You should have now completed a resynchronization of power traces. This is a very useful tool, and you can see how making a simple class could extend this work.

## Tests

In [ ]:
key = project.traceManager().getKnownKey(0)
recv_key = [kguess[0][0] for kguess in attack_results.findMaximums()]
assert (key == recv_key).all(), "Failed to recover encryption key\nGot: {}\nExpected: {}".format(recv_key, key)

In [ ]:
assert (attack_results.pge == [0]*16), "PGE for some bytes not zero: {}".format(attack_results.pge)

In [ ]:
max_corrs = [kguess[0][2] for kguess in attack_results.findMaximums()]
assert (np.all([corr > 0.75 for corr in max_corrs])), "Low correlation in attack (corr <= 0.75): {}".format(max_corrs)